In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def pose_to_matrix(pose):
    """Convert R, t pose to 4x4 transformation matrix"""
    matrix = np.eye(4)
    matrix[:3, :3] = np.array(pose['R'])
    matrix[:3, 3] = np.array(pose['t'])
    return matrix

# Input data
poses = [
    {
        "R": [[1,0,0],[0,1,0],[0,0,1]],
        "t": [0,0,0]
    },
    {
        "R": [[-0.0008290000610233772,-0.7947131755287576,0.6069845808584402],
              [0.7624444396180684,0.3922492478955913,0.5146056781855716],
              [-0.6470531579819294,0.46321862674804054,0.6055994671226776]],
        "t": [-2.6049886186449047,-2.173986915510569,0.7303458563542193]
    },
    {
        "R": [[-0.9985541623963866,-0.028079891357569067,-0.045837806036037466],
              [-0.043210651917521686,-0.08793122558361385,0.9951888962042462],
              [-0.03197537054848707,0.995730696156702,0.0865907408997996]],
        "t": [0.8953888630067902,-3.4302652822708373,3.70967106300893]
    },
    {
        "R": [[-0.4499864100408215,0.6855400696798954,-0.5723172578577878],
              [-0.7145273934510732,0.10804105689305427,0.6912146801345055],
              [0.5356891214002657,0.7199735709654319,0.4412201517663212]],
        "t": [2.50141072072536,-2.313616767292231,1.8529907514099284]
    }
]

to_world_matrix = np.array([
    [0.9941338485260931,0.0986512964608827,-0.04433748889242502,0.9938296704767513],
    [-0.0986512964608827,0.659022672138982,-0.7456252673517598,2.593331619023365],
    [0.04433748889242498,-0.7456252673517594,-0.6648888236128887,2.9576262456228286],
    [0,0,0,1]
])

# Transform each pose
transformed_positions = []
for pose in poses:
    # Convert pose to matrix
    pose_matrix = pose_to_matrix(pose)
    
    # Transform pose
    transformed_matrix = to_world_matrix @ pose_matrix
    
    # Extract position
    position = transformed_matrix[:3, 3]
    transformed_positions.append(position)

transformed_positions = np.array(transformed_positions)

# Calculate average Z coordinate
avg_z = np.mean(transformed_positions[:, 2])
print(f"Average Z coordinate: {avg_z:.4f}")

# Project all points to average Z plane
projected_positions = transformed_positions.copy()
projected_positions[:, 2] = avg_z

# Calculate deviations from average Z plane
deviations = np.abs(transformed_positions[:, 2] - avg_z)
max_deviation = np.max(deviations)
print(f"\nMaximum deviation from average Z plane: {max_deviation:.4f}")

print("\nOriginal transformed positions:")
for i, pos in enumerate(transformed_positions):
    print(f"Camera {i}: {pos} (deviation from avg Z: {deviations[i]:.4f})")

print("\nProjected positions (on average Z plane):")
for i, pos in enumerate(projected_positions):
    print(f"Camera {i}: {pos}")

# Visualize the camera positions
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot original positions
ax.scatter(transformed_positions[:, 0], 
          transformed_positions[:, 1], 
          transformed_positions[:, 2], 
          c='blue', marker='o', label='Original')

# Plot projected positions
ax.scatter(projected_positions[:, 0], 
          projected_positions[:, 1], 
          projected_positions[:, 2], 
          c='red', marker='^', label='Projected')

# Draw lines between original and projected positions
for orig, proj in zip(transformed_positions, projected_positions):
    ax.plot([orig[0], proj[0]], 
            [orig[1], proj[1]], 
            [orig[2], proj[2]], 
            'k--', alpha=0.3)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
plt.title('Camera Positions: Original vs Projected')
plt.show()

# Additional analysis: Calculate the plane normal
def fit_plane(points):
    """Fit a plane to the points and return the normal vector"""
    centroid = np.mean(points, axis=0)
    centered_points = points - centroid
    U, S, Vt = np.linalg.svd(centered_points)
    normal = Vt[-1]
    return normal / np.linalg.norm(normal)

# Fit plane to original positions
plane_normal = fit_plane(transformed_positions)
print("\nBest-fit plane normal vector:", plane_normal)

# Calculate angle between best-fit plane normal and Z-axis
z_axis = np.array([0, 0, 1])
angle = np.arccos(np.dot(plane_normal, z_axis))
print(f"Angle between best-fit plane and XY plane: {np.degrees(angle):.2f} degrees")

In [ ]:
import cv2
import numpy as np
import pycolmap
from pathlib import Path

class IRMarkerDetector:
    def __init__(self, threshold=127):
        self.threshold = threshold
        
    def detect(self, frame):
        # Basic binary threshold for IR bright spots
        _, binary = cv2.threshold(frame, self.threshold, 255, cv2.THRESH_BINARY)
        
        # Denoise if needed
        denoised = cv2.fastNlMeansDenoising(binary)
        
        # Find centroids of IR LEDs
        contours, _ = cv2.findContours(denoised, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        points = []
        for contour in contours:
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                points.append((cX, cY))
                
        return points, binary  # Return both points and processed image for visualization

class CameraPoseEstimator:
    def __init__(self, camera_params, marker_world_coords):
        """
        camera_params: dict with camera intrinsics
        marker_world_coords: nx3 array of real-world coordinates of IR LED markers
        """
        self.camera_matrix = np.array([
            [camera_params['fx'], 0, camera_params['cx']],
            [0, camera_params['fy'], camera_params['cy']],
            [0, 0, 1]
        ])
        self.dist_coeffs = np.array(camera_params['dist_coeffs'])
        self.marker_world_coords = marker_world_coords
        
    def estimate_pose(self, image_points):
        """Returns camera position and rotation matrix"""
        success, rvec, tvec = cv2.solvePnP(
            self.marker_world_coords,
            np.array(image_points),
            self.camera_matrix,
            self.dist_coeffs
        )
        
        if not success:
            return None, None
            
        # Convert rotation vector to matrix
        R, _ = cv2.Rodrigues(rvec)
        
        return R, tvec

In [ ]:
def visualize_camera_pose(frame, R, t, points_2d, processed_binary):
    # Create visualization window
    viz_original = cv2.resize(frame, (640, 480))
    viz_processed = cv2.resize(processed_binary, (640, 480))
    
    # Draw detected points
    for point in points_2d:
        cv2.circle(viz_original, point, 5, (0, 255, 0), -1)
    
    # Combine visualizations
    viz = np.hstack([viz_original, viz_processed])
    
    # Add pose information as text
    position = -np.matrix(R).T * np.matrix(t)
    cv2.putText(viz, f"Position: {position.flatten()}", 
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    return viz

# Main loop
def calibration_tool():
    cap = cv2.VideoCapture(0)  # or your video file
    detector = IRMarkerDetector()
    
    # Define your known IR LED positions (example)
    marker_positions = np.array([
        [0, 0, 0],
        [0, 100, 0],
        [100, 0, 0],
        [100, 100, 0]
    ])  # Replace with your actual marker arrangement
    
    # Camera parameters (you'll need to calibrate these first)
    camera_params = {
        'fx': 1000,
        'fy': 1000,
        'cx': 320,
        'cy': 240,
        'dist_coeffs': [0, 0, 0, 0, 0]
    }
    
    estimator = CameraPoseEstimator(camera_params, marker_positions)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Detect IR markers
        points_2d, processed = detector.detect(frame)
        
        if len(points_2d) >= 4:  # Need at least 4 points for reliable pose estimation
            R, t = estimator.estimate_pose(points_2d)
            if R is not None:
                viz = visualize_camera_pose(frame, R, t, points_2d, processed)
                cv2.imshow('Calibration Tool', viz)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()